# EMOCHAT
**Team** <br>
Amit Krishna A - 19BCE0197 <br>
Swamitha Gupta - 19BCE0728 <br>
Arush Bhat     - 19BCE0564 <br>
Agniva Basak   - 19BCE0744


# Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import *

import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras import backend as K

In [ ]:
import os
import PIL
import PIL.Image
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tqdm import tqdm
import cv2
import pathlib

# Load Dataset 

In [ ]:
dataset = '../input/fer2013/fer2013.csv'

# This dataset consists of 35887 grayscale, 48x48 sized face images with 7 labeled emotions

labels = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']
df=pd.read_csv(dataset,names=names, na_filter=False)
images=df['pixels']
df.head(10)

# Face Detection

In [ ]:
def detect_and_crop_faces(img):
    img = img.astype("uint8")
    face_cascade = cv2.CascadeClassifier("../input/haarcascade-frontalface-default/haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(img, 1.01, 5)
    if len(faces) == 0:
        haar_img = img
    else:
        x, y, w, h = faces[0]
        haar_img = img[y:y+h, x:x+w]
        
    return ((haar_img).astype("float32"))

In [ ]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X = np.array(X)
    Y = np.array(Y)
    return X, Y

In [ ]:
X, Y = getData(dataset)
N, D = X.shape
img_X = X.reshape(N, 48, 48, 1)

In [ ]:
Z = []
for img in tqdm(list(img_X)):
    img_resize = cv2.resize(detect_and_crop_faces(img), (48, 48))
    Z.append(img_resize)
    
X = Z

In [ ]:
X[0].shape

In [ ]:
#Normalization
X = np.array(Z)
X = X/255

num_class = len(set(Y))
print(num_class)

In [ ]:
imgplot = plt.imshow(X[10])

# Training Model

In [ ]:
X = X.reshape(N, 48, 48, 1)
Y = (np.arange(num_class) == Y[:, None]).astype(np.float32)

In [ ]:
print(Y.shape)

In [ ]:
VGG_model = tf.keras.applications.VGG16(include_top=False,weights='imagenet')
bottleneck_features = np.empty([N, 512])

for i in tqdm(range(N)):
    img = np.expand_dims(X[i], axis=0)
    feature = VGG_model.predict(img)
    features_flatten = feature.flatten()
    bottleneck_features[i] = feature.flatten()

In [ ]:
def my_model():
    model = Sequential()
    model.add(VGG_model)
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(num_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    
    return model

model = my_model()
model.summary()    

In [ ]:
def my_model():
    model = Sequential()
    input_shape = (48,48,1)
    
    model.add(Conv2D(64, (5, 5),input_shape = input_shape, activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    
    return model
model=my_model()
model.summary()

In [ ]:
path_model='model_filter.h5' # save model at this location after each epoch
tf.keras.backend.clear_session() # destroys the current graph and builds a new one

model=my_model() # create the model
K.set_value(model.optimizer.lr,1e-3) # set the learning rate

# fit the model
h=model.fit(x=X,     
            y=Y,
            validation_split = 0.2,
            batch_size=64, 
            epochs=20, 
            verbose=1,
            shuffle=True,
            callbacks=[ModelCheckpoint(filepath=path_model),]
            )

In [ ]:
plt.plot(h.history['accuracy'])

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

# Prediction

In [ ]:
objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
y_pos = np.arange(len(objects))
print(y_pos)

In [ ]:
def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
plt.show()

In [ ]:
from skimage import io


def predict_emotion(path):
    img = image.load_img(path, grayscale=True, target_size=(48, 48))
    show_img=image.load_img(path, grayscale=False, target_size=(200, 200))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)

    x /= 255

    custom = model.predict(x)
    emotion_analysis(custom[0])

    x = np.array(x, 'float32')
    x = x.reshape([48, 48]);

    plt.gray()
    plt.imshow(show_img)
    plt.show()

    m=0.000000000000000000001
    a=custom[0]
    for i in range(0,len(a)):
        if a[i]>m:
            m=a[i]
            ind=i
        
    print('Expression Prediction:',objects[ind])

In [ ]:
predict_emotion('../input/images/h_image.jpg')

In [ ]:
predict_emotion('../input/images/s_image.jpg')